# Couette flow simulations

In [1]:
%matplotlib widget 
# global import
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
# local import
import lppydsmc as ld

## General theory

In the next sections, we will study the couette flow for three Knudsen number: 0.1, 1.0 and 10.

The idea first is to compare to the results obtained by Bird (Molecular Gas Dynamic, 1994, section 12.2)


In [2]:
sample_per_cell = 1.3e6
dt = 2.5e-5 # s
factor = 4
sampling_time = 0.275 # sampling_time
nb_iterations = sampling_time/dt
nb_iterations_sampled = nb_iterations/factor
mnpc = sample_per_cell/nb_iterations_sampled
print(nb_iterations) # the total number is 15000. Bird wait 0.1s before sampling, and he take a sample every 4 dt.
print(mnpc)

11000.0
472.72727272727275


## Case 1 

In [3]:
path_couette_1 = "/home/calot/Documents/projets/lppydsmc/benchmarks/config_files/couette_1.ini"

In [4]:
# launching simulations
simulation = False
if(simulation):
    ld.main(path_couette_1, save = True); 
    # seems to be taking its time right now ... May be I should run it at night?

### Loading Results

In [5]:
results_path = "/home/paul/Documents/lppydsmc/benchmarks/results/couette_1_2/monitoring.h5" # seems ok ...
store = pd.HDFStore(results_path)
print(store.keys())

['/dsmc_collisions', '/dsmc_tracking', '/particles', '/wall_collisions']


In [6]:
h = 1e-3
l = 2.5e-5
dy = h/100.
dx = 2.5e-5
U = 300 # m/s
dt = 1e-7
dz = 1e-3

mass = 2.16e-25 # kg
volume_cell = dy*dx*dz
particles_weight = 5000000000
injection_density=2e+24

In [7]:
df_particles = store['particles']

In [8]:
df_particles['v2'] = df_particles['vx']*df_particles['vx']+df_particles['vy']*df_particles['vy']+df_particles['vz']*df_particles['vz']
df_particles['v'] = np.sqrt(df_particles['v2'])

In [9]:
frames = 0.2 # taking the last 80% of the simulation
max_frame = np.max(df_particles.index)
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

In [10]:
df_particles = pd.DataFrame(df_partial)

### Scatter plot

In [53]:
df_last_frame = df_partial.loc[df_partial.index == max_frame]
fig, ax = plt.subplots(constrained_layout=True)
ax.axis('equal')
sns.scatterplot(data = df_last_frame, x='x',y='y');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Distribution

In [ ]:
bins = 100

fig, ax = plt.subplots(2,2, constrained_layout = True)
ax[0,0].hist(df_partial['vx'], bins = bins, color = 'r', density = False)
ax[0,1].hist(df_partial['vy'], bins = bins, color = 'g', density = False)
ax[1,0].hist(df_partial['vz'], bins = bins, color = 'b', density = False)
ax[1,1].hist(df_partial['v'], bins = bins, color = 'k', density = False);

ax[0,0].set_xlabel('vx (m/s)')
ax[0,1].set_xlabel('vy (m/s)')
ax[1,0].set_xlabel('vz (m/s)')
ax[1,1].set_xlabel('v (m/s)');

### Distribution $v_x$ depending on $y$

In [13]:
df_particles['vx/U'] = df_particles['vx']/U
df_particles['y/h'] = df_particles['y']/h
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

In [ ]:
fig, ax = plt.subplots(constrained_layout = True)
sns.histplot(data = df_partial, x = 'vx/U', y = 'y/h', cbar = True, bins = 40, stat = 'density', ax = ax);

### Profile - and dependance on the x-position

In [15]:
df_particles['y_int'] = (df_particles['y']/dy).astype(int) 
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

In [69]:
def plot_profile(ax, df, height, lenght, dx, dy, wall_speed, factor = 0):
    # df['y_int'] = (df['y']/dy).astype(int) 
    if(factor == 0):
        results = df.groupby('y_int').mean()['vx']
    else:
        df_partial_x_low = df.loc[df['x']<factor*dx]
        df_partial_x_mid = df.loc[(df['x']<lenght-factor*dx) & (df_partial['x']>factor*dx)]
        df_partial_x_high = df.loc[df['x']>lenght-factor*dx]
        results_low = df_partial_x_low.groupby('y_int').mean()['vx']
        results_mid = df_partial_x_mid.groupby('y_int').mean()['vx']
        results_high = df_partial_x_high.groupby('y_int').mean()['vx']
        
    ax.set_xlabel('vx/U')
    ax.set_ylabel('y/h')
    if(factor == 0):
        ax.plot(results.values/wall_speed, results.index*dy/height, '.',color = 'k',  label = 'average')
    else:
        ax.plot(results_low.values/wall_speed, results_low.index*dy/height,'.', color = 'r', label = 'x < {:.3}'.format(factor*dx/lenght))
        ax.plot(results_mid.values/wall_speed, results_mid.index*dy/height, '.',color = 'g',  label = '{:.3} < x < {:.3}'.format(factor*dx/lenght, 1-factor*dx/lenght))
        ax.plot(results_high.values/wall_speed, results_high.index*dy/height,'.', color = 'b',  label = '{:.3} < x'.format(1-factor*dx/lenght))
    # ax.legend(loc = 'lower right')
    return ax

In [70]:
fig, ax = plt.subplots(constrained_layout = True)
plot_profile(ax, df_partial, height = h, lenght = l, dx = dx, dy = dy, wall_speed = U, factor = 1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Prolonging profile near the top wall

In [71]:
def prolong_profile(profile, values, height):
    dy = values[1]-values[0]
    first_der = (profile[1:]-profile[:profile.shape[0]-1])/dy
    second_der = (first_der[1:]-first_der[:first_der.shape[0]-1])/dy
    
    dy_end = (height-values[-1])
    prolong_first_order = profile[-1]+dy_end*first_der[-1]
    prolong_second_order = prolong_first_order+0.5*dy_end*dy_end*second_der[-1]
    
    return values[-1]+dy_end, prolong_first_order, prolong_second_order

In [76]:
results = df_partial.groupby('y_int').mean()['vx']

print(prolong_profile(results.values,  results.index.values*dy, h)) # y value, first prolong, 2nd one

(0.001, 284.9396610042644, 284.9210984684353)


### Dependance of the profile on time

We may not be in steady flow yet.

In [16]:
def plot_profile_evolution(ax, df, height, lenght, dx, dy, wall_speed, time_step, number_of_splits = 1):
    s = df.shape[0]
    number_of_frames_per_curve = s//number_of_splits
    max_index = np.max(df.index.values)
    number_of_true_frames = max_index//number_of_splits
    for k in range(number_of_splits):
        df_ = df[k*number_of_frames_per_curve:(k+1)*number_of_frames_per_curve]
        results = df_.groupby('y_int').mean()['vx']
        ax.plot(results.values/wall_speed, results.index*dy/height,  label = '{:.1e} < t < {:.1e} s'.format(time_step*k*number_of_true_frames, time_step*(k+1)*number_of_true_frames))

    ax.set_xlabel('vx/U')
    ax.set_ylabel('y/h')
    ax.legend(loc = 'lower right')


In [17]:
fig, ax = plt.subplots(constrained_layout = True)
plot_profile_evolution(ax, df_particles, height = h, lenght = l, dx = dx, dy = dy,  wall_speed = U, time_step = dt, number_of_splits = 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Temperatuere profile

In [18]:
def plot_profile_temperature(ax, df, height, mass, factor = 0):    
    ax.set_xlabel('T (K)')
    ax.set_ylabel('y/h')
    
    # speed norm
    df['v2'] = df['vx']*df['vx']+df['vy']*df['vy']+df['vz']*df['vz']
    df['v'] = np.sqrt(df['v2'])

    # drift 
    v_mean = df.groupby(df['y_int']).mean()
    v_mean['drift2'] = v_mean['vx']*v_mean['vx']+v_mean['vy']*v_mean['vy']+v_mean['vz']*v_mean['vz']
    v_mean['drift'] = np.sqrt(v_mean['drift2'])

    # 3/2 k T = 1/2 m (<v²>-|<v>|²)
    temperature = mass/(3.*ld.utils.physics.BOLTZMAN_CONSTANT)*(v_mean['v2']-v_mean['drift2'])
    
    ax.plot(temperature,v_mean['y']/height, '.')
    # ax.legend(loc = 'lower right')
    return ax

In [19]:
fig, ax = plt.subplots(constrained_layout = True)
plot_profile_temperature(ax, df_particles, height = h, mass = mass, factor = 1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
def plot_profile_density(ax, df, height, volume_cell, particles_weight):    
    ax.set_xlabel('density ($m^{-3}$)')
    ax.set_ylabel('y/h')
    
    # drift 
    df_count = df.groupby(df['y_int']).count()    
    df_mean = df.groupby(df['y_int']).mean()    
    ax.plot(df_count['y']/(volume_cell*df.index.unique().shape[0])*particles_weight,df_mean['y']/height,'.')
    # ax.legend(loc = 'lower right')
    return ax

In [21]:
fig, ax = plt.subplots(constrained_layout = True)
plot_profile_density(ax, df_particles, height = h, volume_cell = volume_cell, particles_weight = particles_weight);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Shear stress

Bird gives the formula page 285 : $\tau_{x,y} = - n \bar{(muv)}$.
With :
- $n$ the density.
- $u$ the velocity along x
- $v$ the velocity along y
- $m$ the molecular mass

In [22]:
def plot_shere_stress_profile(ax, df, height, density, mass):    
    ax.set_xlabel('shear stress ($N.m^{-2}$)')
    ax.set_ylabel('y/h')
    
    # drift
    df['shear_stress'] = -density*mass*df['vx']*df['vy']
    df_mean = df.groupby(df['y_int']).mean()    
    ax.plot(df_mean['shear_stress'],df_mean['y']/height,'.')
    # ax.legend(loc = 'lower right')
    return ax

In [23]:
fig, ax = plt.subplots(constrained_layout = True)
plot_shere_stress_profile(ax, df_particles, h, injection_density, mass)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='shear stress ($N.m^{-2}$)', ylabel='y/h'>